# Deliverable 1. Retrieve Weather Data.
---
1. Create a folder called `Weather_Database` to save all the files related with this deliverable.

2. Save the `Weather_Database_starter_code.ipynb` starter code to the `Weather_Database` folder and rename it as `Weather_Database.ipynb`.

3. Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

4. Use the `citipy` module to get the nearest city for each latitude and longitude combination.

5. Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key; also, it's critical to avoid publishing your API key on your GitHub repository.

6. Retrieve the following information from the API call:

    * Latitude and longitude

    * Maximum temperature

    * Percent humidity

    * Percent cloudiness

    * Wind speed

    * Weather description (for example, clouds, fog, light rain, clear sky)

7. Add the weather data to a new DataFrame.

8. Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.
---

In [26]:
# Import initial libraries
import numpy as np
import pandas as pd
from citipy import citipy
# Dependencies
import requests
import json

## Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

In [27]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)

# Use the zip function to create an iterator of tuples containing the latitude and longitude combinations
lat_lngs = zip(lats, lngs) #output is a zip object in memory. # packs into a tuple.
lat_lngs

In [28]:
# Add the latitudes and longitudes to a list
coordinates = lat_lngs

## Use the `citipy` module to get the nearest city for each latitude and longitude combination.

In [29]:
# Create an empty list for holding the cities
cities = []

# Use a for loop to identify nearest city for each latitude and longitude combination using the citipy module
for coordinate in coordinates:
    # Use the citipy module to identify the nearest city to the current coordinate
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

622

## Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key.

In [30]:
# Import the requests library
import requests

# Import the time library
import time

# Import the datetime module from the datetime library
from datetime import datetime

# Import the OpenWeatherMap's API key
from config import weather_api_key

In [31]:
# Assemble the OpenWeatherMap's API call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

## Retrieve the following information from the API call.

- The latitude and longitude
- The Max temperature
- The % humidity
- The % cloudiness
- The Wind speed
- The Weather description, i.e., cloudy, fog, light rain, clear sky, etc. 

In [33]:

 city_url = url + "&q=" + "Denver".replace(" ","+")
weather_data = requests.get(city_url).json()
print(json.dumps(weather_data, indent=4, sort_keys=True))

{
    "base": "stations",
    "clouds": {
        "all": 53
    },
    "cod": 200,
    "coord": {
        "lat": 39.7392,
        "lon": -104.9847
    },
    "dt": 1669310598,
    "id": 5419384,
    "main": {
        "feels_like": 35.06,
        "humidity": 67,
        "pressure": 1031,
        "temp": 35.06,
        "temp_max": 38.5,
        "temp_min": 32.18
    },
    "name": "Denver",
    "sys": {
        "country": "US",
        "id": 2004334,
        "sunrise": 1669298090,
        "sunset": 1669333133,
        "type": 2
    },
    "timezone": -25200,
    "visibility": 10000,
    "weather": [
        {
            "description": "broken clouds",
            "icon": "04d",
            "id": 803,
            "main": "Clouds"
        }
    ],
    "wind": {
        "deg": 286,
        "gust": 5.01,
        "speed": 1.99
    }
}


In [34]:
weather_data["weather"][0]["description"]

'broken clouds'

In [36]:
# Create an empty list to hold weather data for each city
city_data = []

# Print a message to indicate that the data retrieval starts
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters and set them to 1
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data for each city
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create an endpoint URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Description": description})
    
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

# Indicate that the data retrieval is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | hermanus
Processing Record 2 of Set 1 | amderma
City not found. Skipping...
Processing Record 3 of Set 1 | mount isa
Processing Record 4 of Set 1 | nizhneyansk
City not found. Skipping...
Processing Record 5 of Set 1 | hambantota
Processing Record 6 of Set 1 | carnarvon
Processing Record 7 of Set 1 | souris
Processing Record 8 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 9 of Set 1 | leningradskiy
Processing Record 10 of Set 1 | honiara
Processing Record 11 of Set 1 | arraial do cabo
Processing Record 12 of Set 1 | shunyi
Processing Record 13 of Set 1 | castro
Processing Record 14 of Set 1 | vila
Processing Record 15 of Set 1 | yellowknife
Processing Record 16 of Set 1 | husavik
Processing Record 17 of Set 1 | barcelona
Processing Record 18 of Set 1 | busselton
Processing Record 19 of Set 1 | hobart
Processing Record 20 of Set 1 | thompson
Processing Record 21 of Set 1 | mal

Processing Record 35 of Set 4 | bethel
Processing Record 36 of Set 4 | bilibino
Processing Record 37 of Set 4 | faanui
Processing Record 38 of Set 4 | kapaa
Processing Record 39 of Set 4 | twentynine palms
Processing Record 40 of Set 4 | kenai
Processing Record 41 of Set 4 | thinadhoo
Processing Record 42 of Set 4 | dingle
Processing Record 43 of Set 4 | antalaha
Processing Record 44 of Set 4 | le port
Processing Record 45 of Set 4 | gamba
Processing Record 46 of Set 4 | cabedelo
Processing Record 47 of Set 4 | esperance
Processing Record 48 of Set 4 | cockburn town
Processing Record 49 of Set 4 | bargal
City not found. Skipping...
Processing Record 50 of Set 4 | marsh harbour
Processing Record 1 of Set 5 | alanya
Processing Record 2 of Set 5 | loksa
Processing Record 3 of Set 5 | urumqi
Processing Record 4 of Set 5 | serebryanyy bor
Processing Record 5 of Set 5 | flinders
Processing Record 6 of Set 5 | umzimvubu
City not found. Skipping...
Processing Record 7 of Set 5 | batticaloa
Pro

Processing Record 1 of Set 9 | fairview
City not found. Skipping...
Processing Record 2 of Set 9 | kerki
City not found. Skipping...
Processing Record 3 of Set 9 | san cristobal
City not found. Skipping...
Processing Record 4 of Set 9 | port macquarie
City not found. Skipping...
Processing Record 5 of Set 9 | plastun
City not found. Skipping...
Processing Record 6 of Set 9 | turukhansk
City not found. Skipping...
Processing Record 7 of Set 9 | cayenne
City not found. Skipping...
Processing Record 8 of Set 9 | hun
City not found. Skipping...
Processing Record 9 of Set 9 | bokspits
City not found. Skipping...
Processing Record 10 of Set 9 | colesberg
City not found. Skipping...
Processing Record 11 of Set 9 | amga
City not found. Skipping...
Processing Record 12 of Set 9 | denpasar
City not found. Skipping...
Processing Record 13 of Set 9 | kalmunai
City not found. Skipping...
Processing Record 14 of Set 9 | fernie
City not found. Skipping...
Processing Record 15 of Set 9 | saint-georges

Processing Record 30 of Set 11 | lagunas
Processing Record 31 of Set 11 | chiredzi
Processing Record 32 of Set 11 | el alto
Processing Record 33 of Set 11 | murmashi
Processing Record 34 of Set 11 | comodoro rivadavia
Processing Record 35 of Set 11 | geresk
City not found. Skipping...
Processing Record 36 of Set 11 | griffith
Processing Record 37 of Set 11 | buala
Processing Record 38 of Set 11 | punto fijo
Processing Record 39 of Set 11 | afmadu
City not found. Skipping...
Processing Record 40 of Set 11 | baruun-urt
Processing Record 41 of Set 11 | hampton
Processing Record 42 of Set 11 | kouroussa
Processing Record 43 of Set 11 | novyy urengoy
Processing Record 44 of Set 11 | hofn
Processing Record 45 of Set 11 | henties bay
Processing Record 46 of Set 11 | boo
Processing Record 47 of Set 11 | jiaocheng
Processing Record 48 of Set 11 | billings
Processing Record 49 of Set 11 | phalaborwa
Processing Record 50 of Set 11 | yingzhong
City not found. Skipping...
Processing Record 1 of Set

In [37]:
# Print the length of the city_data list to verify how many cities you have
len(city_data)

434

## Add the weather data to a new DataFrame.

In [38]:
# Use the city_data list to create a new pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

In [39]:
# Display sample data
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,Hermanus,-34.4187,19.2345,60.80,56,88,10.27,ZA,2022-11-24 17:42:48,overcast clouds
1,Mount Isa,-20.7333,139.5000,74.97,38,40,4.61,AU,2022-11-24 17:42:48,scattered clouds
2,Hambantota,6.1241,81.1185,76.77,85,80,8.52,LK,2022-11-24 17:42:49,broken clouds
3,Carnarvon,-24.8667,113.6333,68.16,75,0,20.71,AU,2022-11-24 17:42:49,clear sky
4,Souris,49.6172,-100.2612,26.46,86,96,9.93,CA,2022-11-24 17:42:50,overcast clouds
5,Leningradskiy,69.3833,178.4167,6.13,95,100,7.83,RU,2022-11-24 17:42:50,light snow
6,Honiara,-9.4333,159.9500,77.22,86,7,4.47,SB,2022-11-24 17:42:51,clear sky
7,Arraial Do Cabo,-22.9661,-42.0278,73.36,94,75,6.91,BR,2022-11-24 17:42:51,light rain
8,Shunyi,40.1250,116.6453,42.75,100,100,2.24,CN,2022-11-24 17:42:52,mist
9,Castro,-24.7911,-50.0119,74.08,57,65,5.28,BR,2022-11-24 17:42:52,broken clouds


In [40]:
# Display the DataFrame's column names using the columns Pandas function
city_data_df.columns

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date', 'Description'],
      dtype='object')

In [42]:
# Create a list to reorder the column names as follows:
# "City", "Country", "Lat", "Lng", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Current Description"
new_column_order = ["City", "Country", "Date", 'Lat', "Lng","Max Temp", "Humidity","Cloudiness","Wind Speed", "Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,Hermanus,ZA,2022-11-24 17:42:48,-34.4187,19.2345,60.80,56,88,10.27,overcast clouds
1,Mount Isa,AU,2022-11-24 17:42:48,-20.7333,139.5000,74.97,38,40,4.61,scattered clouds
2,Hambantota,LK,2022-11-24 17:42:49,6.1241,81.1185,76.77,85,80,8.52,broken clouds
3,Carnarvon,AU,2022-11-24 17:42:49,-24.8667,113.6333,68.16,75,0,20.71,clear sky
4,Souris,CA,2022-11-24 17:42:50,49.6172,-100.2612,26.46,86,96,9.93,overcast clouds
5,Leningradskiy,RU,2022-11-24 17:42:50,69.3833,178.4167,6.13,95,100,7.83,light snow
6,Honiara,SB,2022-11-24 17:42:51,-9.4333,159.9500,77.22,86,7,4.47,clear sky
7,Arraial Do Cabo,BR,2022-11-24 17:42:51,-22.9661,-42.0278,73.36,94,75,6.91,light rain
8,Shunyi,CN,2022-11-24 17:42:52,40.1250,116.6453,42.75,100,100,2.24,mist
9,Castro,BR,2022-11-24 17:42:52,-24.7911,-50.0119,74.08,57,65,5.28,broken clouds


In [43]:
# Display the data types of each column by using the dtypes Pandas function
city_data_df.dtypes

City            object
Country         object
Date            object
Lat            float64
Lng            float64
Max Temp       float64
Humidity         int64
Cloudiness       int64
Wind Speed     float64
Description     object
dtype: object

## Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.

In [44]:
# Create the output file (CSV).
output_data_file = "Weather_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")